In [1]:
# Import statements
import pandas as pd
import numpy as np


In [18]:
# HPI Data
df = pd.read_csv('datasets/hpi_master.csv')
cities = ["Houston", "Phoenix", "Los Angeles", "Chicago", "New York"]
pattern = "|".join(cities)
filtered_df = df[df['place_name'].str.contains(pattern, case=False, na=False)]
filtered_df = filtered_df[filtered_df['hpi_type'] == 'traditional']
filtered_df = filtered_df[filtered_df['hpi_flavor'] == 'all-transactions']
filtered_df = filtered_df[filtered_df['level'] == 'MSA']
filtered_df = filtered_df.drop(columns=['hpi_type', 'hpi_flavor', 'frequency', 'level', 'place_id', 'index_sa'])
def extract_city(name):
    for city in cities:
        if city.lower() in name.lower():
            return city
    return name

filtered_df['place_name'] = filtered_df['place_name'].apply(extract_city)
filtered_df = filtered_df[filtered_df['yr'] >= 1990]
filtered_df = filtered_df.rename(columns={'yr': 'Year', 'place_name': 'City', 'period': 'Quarter'})
filtered_df = filtered_df.drop_duplicates()
hpi = filtered_df
hpi.head()

,City,Year,Quarter,index_nsa
16354,Chicago,1990,1,83.83
16355,Chicago,1990,2,84.81
16356,Chicago,1990,3,85.80
16357,Chicago,1990,4,85.87
16358,Chicago,1991,1,86.95


In [19]:
#Demographic Data
import pandas as pd
import numpy as np

df_income = pd.read_excel('datasets/income_data.xlsx', sheet_name='Sheet1')
df_age_before = pd.read_excel('datasets/less_age_data.xlsx', sheet_name='Sheet1')

df_less_income = df_income[['Median Household Income', 'City', 'Year']]

data = {
    'Median Household Income': [38909, 26301, 28327, 40328, 45600, 38293, 37625, 41207, 36616, 36687],
    'City': ['New York', 'Chicago', 'Phoenix', 'Houston', 'Los Angeles', 'New York', 'Chicago', 'Phoenix', 'Houston', 'Los Angeles'],
    'Year': [1990, 1990, 1990, 1990, 1990, 2000, 2000, 2000, 2000, 2000],
}

df_income_before = pd.DataFrame(data)

df_income_combined = pd.concat([df_less_income, df_income_before], axis=0)



df_income_combined = df_income_combined.sort_values(by=['City', 'Year']).reset_index(drop=True)
df_income_combined['Date'] = pd.to_datetime(df_income_combined['Year'], format='%Y') + pd.offsets.QuarterEnd(4)

quarters = pd.date_range(start='1990-03-31', end='2022-12-31', freq='Q')

median_income_quarterly = []

for city, group in df_income_combined.groupby('City'):
    city_df = group.set_index('Date').sort_index()
    city_quarterly = pd.DataFrame(index=quarters)
    city_quarterly = city_quarterly.join(city_df['Median Household Income'], how='left')
    city_quarterly['Median Household Income'] = city_quarterly['Median Household Income'].interpolate(method='linear')
    city_quarterly['Median Household Income'] = city_quarterly['Median Household Income'].ffill().bfill()
    city_quarterly['City'] = city
    median_income_quarterly.append(city_quarterly)

df_income_combined = pd.concat(median_income_quarterly).reset_index().rename(columns={'index': 'Date'})
df_income_combined['Year'] = df_income_combined['Date'].dt.year
df_income_combined['Quarter'] = df_income_combined['Date'].dt.quarter
df_income_combined = df_income_combined[['Median Household Income', 'City', 'Year', 'Quarter']]


# Population Section

age_data_combined = df_age_before.sort_values(by=['City', 'Year']).reset_index(drop=True)
age_data_combined['Date'] = pd.to_datetime(age_data_combined['Year'], format='%Y') + pd.offsets.QuarterEnd(4)

median_age_quarterly = []
for city, group in age_data_combined.groupby('City'):
    city_df = group.set_index('Date').sort_index()
    city_quarterly = pd.DataFrame(index=quarters)
    city_quarterly = city_quarterly.join(city_df['Total Population'], how='left')
    city_quarterly['Total Population'] = city_quarterly['Total Population'].interpolate(method='linear')
    city_quarterly['Total Population'] = city_quarterly['Total Population'].ffill().bfill()
    city_quarterly['City'] = city
    median_age_quarterly.append(city_quarterly)


df_age_combined = pd.concat(median_age_quarterly).reset_index().rename(columns={'index': 'Date'})
df_age_combined['Year'] = df_age_combined['Date'].dt.year
df_age_combined['Quarter'] = df_age_combined['Date'].dt.quarter
df_age_combined = df_age_combined[['Total Population', 'City', 'Year', 'Quarter']]

# Merging Section
final_combined = pd.merge(df_income_combined, df_age_combined, on=['City', 'Year', 'Quarter'], how='inner')
final_combined = final_combined.sort_values(by=['City', 'Year', 'Quarter']).reset_index(drop=True)
#final_combined.to_excel('datasets/final_combined_data.xlsx', index=False)
demographic = final_combined
demographic.tail()


C:\Users\katel\AppData\Local\Temp\ipykernel_46828\2520901345.py:25: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  quarters = pd.date_range(start='1990-03-31', end='2022-12-31', freq='Q')


,Median Household Income,City,Year,Quarter,Total Population
655,68435.0,Phoenix,2021,4,1624539.0
656,70318.5,Phoenix,2022,1,1629505.0
657,72202.0,Phoenix,2022,2,1634471.0
658,74085.5,Phoenix,2022,3,1639437.0
659,75969.0,Phoenix,2022,4,1644403.0


In [20]:
# Industrial production
df = pd.read_csv('datasets/industrial_production.csv')
df['observation_date'] = pd.to_datetime(df['observation_date'])
df['year'] = df['observation_date'].dt.year
df = df[df['year'] >= 1990]
df['month'] = df['observation_date'].dt.month
df['quarter'] = ((df['month'] - 1) // 3) + 1
df = df.drop(columns=['month'])
quarterly_avg = df.groupby(['year', 'quarter']).mean(numeric_only=True).reset_index()
cities = ["Houston", "Phoenix", "Los Angeles", "Chicago", "New York"]
quarterly_avg_expanded = pd.DataFrame(
    quarterly_avg.loc[quarterly_avg.index.repeat(len(cities))].reset_index(drop=True)
)
quarterly_avg_expanded['City'] = cities * len(quarterly_avg)
industrial_production = quarterly_avg_expanded
industrial_production = industrial_production.rename(columns={'year': 'Year', 'quarter': 'Quarter'})
industrial_production.head()

,Year,Quarter,INDPRO,City
0,1990,1,62.1073,Houston
1,1990,1,62.1073,Phoenix
2,1990,1,62.1073,Los Angeles
3,1990,1,62.1073,Chicago
4,1990,1,62.1073,New York


In [21]:
# Volatility Data
df = pd.read_csv('datasets/VIXCLS.csv')
df['observation_date'] = pd.to_datetime(df['observation_date'])

df['VIXCLS'] = df['VIXCLS'].interpolate(method='linear')

df['year'] = df['observation_date'].dt.year
df['quarter'] = df['observation_date'].dt.quarter

quarterly_df = df.groupby(['year', 'quarter'], as_index=False)['VIXCLS'].mean()

cities = ["Houston", "Phoenix", "Los Angeles", "Chicago", "New York"]
vix = pd.DataFrame()

for city in cities:
    temp = quarterly_df.copy()
    temp['city'] = city
    vix = pd.concat([vix, temp], ignore_index=True)

vix = vix[['city', 'year', 'quarter', 'VIXCLS']]
vix = vix.rename(columns={'year': 'Year', 'quarter': 'Quarter', 'city': 'City'})

vix.head()

,City,Year,Quarter,VIXCLS
0,Houston,1990,1,22.168437
1,Houston,1990,2,18.735385
2,Houston,1990,3,25.134462
3,Houston,1990,4,26.005909
4,Houston,1991,1,22.427344


In [22]:
# Inflation Data - CPI 

# Import necessary libraries
import pandas as pd
import numpy as np

# Load CPI data
cpi = pd.read_excel("datasets/consumer-price-inflation.xlsx")

# Drop CPI for all countries except US and unnecessary columns
cpi_us = cpi[cpi['Country'] == 'United States'].copy()
cpi_us.drop(columns=['Country Code', 'IMF Country Code', 'Indicator Type', 'Series Name', 'Note', 'Country'], inplace=True)
cpi_us.head()

# Transpose the dataframe
cpi_us_t = cpi_us.T
cpi_us_t = cpi_us_t.reset_index().rename(columns={'index': 'Year', 190: 'CPI'})

# Splitting years in quarters for CPI
cpi_us_t_quarters = cpi_us_t.loc[cpi_us_t.index.repeat(4)].reset_index(drop=True)
cpi_us_t_quarters['Quarter'] = np.tile([1, 2, 3, 4], len(cpi_us_t))
inflation = cpi_us_t_quarters
cities = ["Houston", "Phoenix", "Los Angeles", "Chicago", "New York"]

inflation_expanded = pd.DataFrame(
    inflation.loc[inflation.index.repeat(len(cities))].reset_index(drop=True)
)
inflation_expanded['City'] = cities * len(inflation)

inflation = inflation_expanded
inflation.head()


,Year,CPI,Quarter,City
0,1970,5.8953,1,Houston
1,1970,5.8953,1,Phoenix
2,1970,5.8953,1,Los Angeles
3,1970,5.8953,1,Chicago
4,1970,5.8953,1,New York


In [23]:
# Unemployment Data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load unemployment data for 5 cities
city_files = {
    "Chicago": "datasets/chicago-unemp.xlsx",
    "Houston": "datasets/houston-unemp.xlsx",
    "Phoenix": "datasets/phoenix-unemp.xlsx",
    "Los Angeles": "datasets/los-angeles-unemp.xlsx",
    "New York": "datasets/new-york-unemp.xlsx"
}

city_dfs = []
for city, file in city_files.items():
    df = pd.read_excel(file)
    df = df[['Year', 'Period', 'unemployment rate']]
    df['City'] = city
    # Convert monthly data to quarter
    def period_to_quarter(period):
        month_to_quarter = {
            'Jan': 1, 'Feb': 1, 'Mar': 1,
            'Apr': 2, 'May': 2, 'Jun': 2,
            'Jul': 3, 'Aug': 3, 'Sep': 3,
            'Oct': 4, 'Nov': 4, 'Dec': 4
        }
        month = period[:3]
        return month_to_quarter.get(month, None)
    df['Quarter'] = df['Period'].apply(period_to_quarter)
    city_dfs.append(df)

# Concatenate all cities into one DataFrame
unemployment_long = pd.concat(city_dfs, ignore_index=True)

# Group by City, Year, Quarter and calculate average unemployment rate
unemployment_grouped = unemployment_long.groupby(['City', 'Year', 'Quarter'], as_index=False)['unemployment rate'].mean()

# Only include data from 1990 onwards
unemployment_grouped = unemployment_grouped[unemployment_grouped['Year'] >= 1990]
unemployment_grouped = unemployment_grouped.rename(columns={'unemployment rate': 'Unemployment Rate'})
unemployment = unemployment_grouped
unemployment.head()

,City,Year,Quarter,Unemployment Rate
0,Chicago,1990,1,6.600000
1,Chicago,1990,2,6.400000
2,Chicago,1990,3,6.466667
3,Chicago,1990,4,6.133333
4,Chicago,1991,1,7.166667


In [24]:
#add national gdp data to the csv file
import pandas as pd

city_file = "datasets/final_combined_data.xlsx"
gdp_file = "datasets/National GDP.csv"

city_df = pd.read_excel(city_file)
gdp_df = pd.read_csv(gdp_file)

gdp_df["DATE"] = pd.to_datetime(gdp_df["observation_date"])
gdp_df["Year"] = gdp_df["DATE"].dt.year
gdp_df["Quarter"] = gdp_df["DATE"].dt.quarter

gdp_quarterly = gdp_df[["Year", "Quarter", "GDP"]].rename(columns={"GDP": "National GDP_Billion"})

merged = city_df.merge(gdp_quarterly, on=["Year", "Quarter"], how="left")
#merged.to_excel("datasets/test_output.xlsx", index=False)



#print(merged.head(20))



In [25]:
import pandas as pd
from sklearn.linear_model import LinearRegression

pd.set_option('display.float_format', '{:,.0f}'.format)

city_gdp_file = "datasets/5cities (1) (1).csv"      
city_file = "datasets/final_combined_data.xlsx"
gdp_file = "datasets/National GDP.csv"

city_df = pd.read_excel(city_file)
gdp_df = pd.read_csv(gdp_file)

gdp_df["DATE"] = pd.to_datetime(gdp_df["observation_date"])
gdp_df["Year"] = gdp_df["DATE"].dt.year
gdp_df["Quarter"] = gdp_df["DATE"].dt.quarter
gdp_quarterly = gdp_df[["Year", "Quarter", "GDP"]].rename(columns={"GDP": "National GDP_Billion"})

quarter_df = city_df.merge(gdp_quarterly, on=["Year", "Quarter"], how="left")

city_gdp_df = pd.read_csv(city_gdp_file)
city_gdp_long = city_gdp_df.melt(
    id_vars=["GeoFips", "GeoName"], 
    var_name="Year", 
    value_name="City GDP"
)
city_gdp_long["Year"] = city_gdp_long["Year"].astype(int)
city_gdp_long.rename(columns={"GeoName": "City"}, inplace=True)

merged = quarter_df.merge(city_gdp_long, on=["City", "Year"], how="left")
merged = merged.drop(columns=["GeoFips"])

train = merged.dropna(subset=["City GDP"])  
X_train = train[["Total Population", "National GDP_Billion"]]
y_train = train["City GDP"]

model = LinearRegression()
model.fit(X_train, y_train)

predict_data = merged[merged["Year"] < 2001]
X_pred = predict_data[["Total Population", "National GDP_Billion"]]
merged.loc[merged["Year"] < 2001, "City GDP"] = model.predict(X_pred)

gdp = merged.drop(columns=["Median Household Income", "Total Population", "National GDP_Billion"])
gdp.head()

,City,Year,Quarter,City GDP
0,Chicago,1990,1,"111,637,782"
1,Chicago,1990,2,"114,954,523"
2,Chicago,1990,3,"117,046,805"
3,Chicago,1990,4,"116,652,451"
4,Chicago,1991,1,"117,896,180"


In [26]:
merged_df = pd.merge(hpi, demographic, on=['Year', 'Quarter', 'City'], how='left')
merged_df = pd.merge(merged_df, industrial_production, on=['Year', 'Quarter', 'City'], how='left')
merged_df = pd.merge(merged_df, vix, on=['Year', 'Quarter', 'City'], how='left')
merged_df = pd.merge(merged_df, inflation, on=['Year', 'Quarter', 'City'], how='left')
merged_df = pd.merge(merged_df, unemployment, on=['Year', 'Quarter', 'City'], how='left')
merged_df = pd.merge(merged_df, gdp, on=['Year', 'Quarter', 'City'], how='left')


final_df = merged_df[(merged_df['Year'] < 2023)]
print(final_df)
final_df.shape

        City  Year  Quarter  index_nsa  Median Household Income  \
0    Chicago  1990        1         84                   26,301   
1    Chicago  1990        2         85                   26,301   
2    Chicago  1990        3         86                   26,301   
3    Chicago  1990        4         86                   26,301   
4    Chicago  1991        1         87                   26,584   
..       ...   ...      ...        ...                      ...   
695  Phoenix  2021        4        436                   68,435   
696  Phoenix  2022        1        458                   70,318   
697  Phoenix  2022        2        495                   72,202   
698  Phoenix  2022        3        502                   74,086   
699  Phoenix  2022        4        478                   75,969   

     Total Population  INDPRO  VIXCLS  CPI  Unemployment Rate    City GDP  
0           2,784,000      62      22    5                  7 111,637,782  
1           2,784,000      63      19    5 

(660, 11)

In [ ]:
# Save merged_df to CSV file
# merged_df.to_csv('final_merged_output.csv', index=False)
# print('merged_df saved to merged_output.csv')

merged_df saved to merged_output.csv
